<a href="https://colab.research.google.com/github/JulianCarax01/Fondamenti-web-app/blob/main/DijkstraSerialCPPMerge_postRicevimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
%%writefile dijkstra_serialMerge.cu
#include <iostream>
#include <vector>
#include <cstdlib>
#include <climits>
#include <cuda.h>
#include <curand_kernel.h>
#include <chrono>

#include <cuda.h>
#include <iostream>
#include <climits>

__device__ void dijkstraSerialDevice(int *graph, int *distances, int *visited, int n, int start) {
    distances[start] = 0;
    visited[start] = 1;

    for (int i = 0; i < n; ++i) {
        int minDist = INT_MAX;
        int currentNode = -1;

        // Trova il nodo con la distanza minima non ancora visitato
        for (int j = 0; j < n; ++j) {
            if (!visited[j] && distances[j] < minDist) {
                minDist = distances[j];
                currentNode = j;
            }
        }

        if (currentNode == -1) break; // Nessun altro nodo raggiungibile

        // Segna il nodo come visitato
        visited[currentNode] = 1;

        // Aggiorna le distanze per i nodi adiacenti
        for (int j = 0; j < n; ++j) {
            if (graph[currentNode * n + j] > 0) {
                int newDist = distances[currentNode] + graph[currentNode * n + j];
                if (newDist < distances[j]) {
                    distances[j] = newDist;
                }
            }
        }
    }
}


__global__ void dijkstraSerialKernel(int *graph, int *distances, int *visited, int n, int start) {
    // La GPU esegue in modalità seriale, quindi un unico thread
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        dijkstraSerialDevice(graph, distances, visited, n, start);
    }
}

void dijkstraSerialCUDA(int *h_graph, int n, int start, int *h_distances) {
    // Cambia float in int ovunque necessario
    int *d_graph, *d_visited;
    int *d_distances;

    cudaMalloc((void **)&d_graph, n * n * sizeof(int));
    cudaMalloc((void **)&d_distances, n * sizeof(int));
    cudaMalloc((void **)&d_visited, n * sizeof(int));

    int *h_distancesInit = new int[n];
    int *h_visitedInit = new int[n];
    for (int i = 0; i < n; i++) {
        h_distancesInit[i] = (i == start) ? 0 : INT_MAX;
        h_visitedInit[i] = 0;
    }

    cudaMemcpy(d_graph, h_graph, n * n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_distances, h_distancesInit, n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_visited, h_visitedInit, n * sizeof(int), cudaMemcpyHostToDevice);

    dijkstraSerialKernel<<<1, 1>>>(d_graph, d_distances, d_visited, n, start);
    cudaDeviceSynchronize();

    cudaMemcpy(h_distances, d_distances, n * sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(d_graph);
    cudaFree(d_distances);
    cudaFree(d_visited);
    delete[] h_distancesInit;
    delete[] h_visitedInit;
}


// Kernel CUDA per aggiornare le distanze
__global__ void cudaNodeRelax(int *graph, int *distances, char *visited, int n, int currentNode) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x; // Stabiliamo l'indice generale del thread tramite le formule viste a lezione

    // Verifichiamo che l'indice sia inferiore al numero totale dei nodi e che il nodo non sia già stato visitato,
    // inoltre facciamo un check per verificare che la distanza tra due nodi non sia zero
    if (idx < n && !visited[idx] && graph[currentNode * n + idx] > 0) {
        int newDist = distances[currentNode] + graph[currentNode * n + idx];
        atomicMin(&distances[idx], newDist); // Aggiorniamo la distanza usando atomicMin per evitare condizioni di gara
    }
    __syncthreads();
}

__global__ void findClosestNodeCUDA(int* d_graph, int d_distances[], char* d_visited, int n){
  //calcolo di thread id per unrolling
  //int tid = blockIdx.x * blockDim.x + threadIdx.x;
  for (int i = 0; i < n; ++i) {

        // Troviamo il nodo non visitato con distanza minima
        int minDist = INT_MAX;
        int currentNode = -1;
        for (int j = 0; j < n; j++) {
            // Controlliamo che il nodo non sia stato visitato e che la distanza sia inferiore a quella nota
            if (!d_visited[j] && d_distances[j] < minDist) {
                minDist = d_distances[j];
                currentNode = j;
            }
        }

        if (currentNode == -1) break; // Se non ci sono più nodi raggiungibili
        d_visited[currentNode] = 1;
/*
        // Aggiorna le distanze in parallelo
        cudaMemcpy(d_distances, distances.data(), n * sizeof(int), cudaMemcpyHostToDevice);
        cudaMemcpy(d_visited, visited.data(), n * sizeof(char), cudaMemcpyHostToDevice);*/

        cudaNodeRelax<<<64, 64>>>(d_graph, d_distances, d_visited, n, currentNode);
        __syncthreads();
/*
        cudaMemcpy(distances.data(), d_distances, n * sizeof(int), cudaMemcpyDeviceToHost);*/
    }
}


// Funzione host per eseguire Dijkstra in CUDA
void dijkstraCUDA(int *graph, int n, int start) {
    // Allocazione memoria su GPU
    int *d_graph; //puntatore memorua gpu
    int *d_distances;
    char *d_visited; // Usiamo un tipo char per verificare che il nodo sia già stato verificato o no (non riesco a farlo con il bool)

    // Utilizziamo la malloc per andare ad allocare sulla memoria
    cudaMalloc((void **)&d_graph, n * n * sizeof(int));
    cudaMalloc((void **)&d_distances, n * sizeof(int));
    cudaMalloc((void **)&d_visited, n * sizeof(char));

    // Inizializza distanze e visited
    std::vector<int> distances(n, INT_MAX);
    std::vector<char> visited(n, 0);
    distances[start] = 0;

    // Copia grafico su GPU
    cudaMemcpy(d_graph, graph, n * n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_distances, distances.data(), n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_visited, visited.data(), n * sizeof(char), cudaMemcpyHostToDevice); // Cambiato da bool a char

    // Esegui Dijkstra iterativamente

    findClosestNodeCUDA <<<1024, 1024>>>(d_graph, d_distances, d_visited, n);

    // Libera memoria GPU
    cudaFree(d_graph);
    cudaFree(d_distances);
    cudaFree(d_visited);

      /* // Stampa la matrice con "X" sulla diagonale
    std::cout << "Matrice del grafo:\n";
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            if (i == j) {
                std::cout << "X "; // Stampa "X" sulla diagonale
            } else {
                std::cout << graph[i * n + j] << " ";
            }
        }
        std::cout << "\n";
    }

    // Stampa solo le distanze alla fine
    std::cout << "Distanze dal nodo iniziale:\n";
    for (int i = 0; i < n; ++i) {
        if (distances[i] == INT_MAX) {
            std::cout << "Nodo " << i << ": NON ESISTE COLLEGAMENTO TRA I NODI\n";
        } else {
            std::cout << "Nodo " << i << ": " << distances[i] << "\n";
        }
    } */

}



__global__ void generateGraphKernel(int *graph, int n, int minWeight, int maxWeight, unsigned int seed) {
    // Calcola l'indice globale del thread
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    // Stato del generatore casuale
    curandState state;

    // Grid-stride loop per coprire tutta la matrice
    for (int i = idx; i < n * n; i += blockDim.x * gridDim.x) {
        int row = i / n; // Calcola la riga
        int col = i % n; // Calcola la colonna

        // Inizializza lo stato casuale
        curand_init(seed + i, 0, 0, &state);

        // Calcola il peso per la triangolare superiore
        int weight = (row < col)
                     ? ((curand(&state) % 100 < 40) ? curand(&state) % (maxWeight - minWeight + 1) + minWeight : 0)
                     : 0;

        // Scrive nella memoria globale
        if (row < col) {
            graph[row * n + col] = weight;
            graph[col * n + row] = weight; // Simmetria
        }

        // Imposta la diagonale a 0
        if (row == col) {
            graph[row * n + col] = 0;
        }
    }
}




void generateGraphCUDA(int *graph, int n, int minWeight, int maxWeight) {
    int *d_graph;
    size_t size = n * n * sizeof(int);

    // Allocazione della memoria sulla GPU
    if (cudaMalloc(&d_graph, size) != cudaSuccess) {
        std::cerr << "Errore nell'allocazione della memoria sulla GPU." << std::endl;
        return;
    }
    if (cudaMemset(d_graph, 0, size) != cudaSuccess) {
        std::cerr << "Errore durante l'inizializzazione della memoria sulla GPU." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Configura blocchi e griglia con valori fissi
    dim3 threadsPerBlock(1024, 1, 1); // 1024 thread per blocco
    dim3 blocksPerGrid(256, 4, 1);   // 1024 blocchi nella griglia

    // Calcola la dimensione della shared memory per blocco
    size_t sharedMemorySize = threadsPerBlock.x * sizeof(int);

    // Lancia il kernel con configurazione fissa
    generateGraphKernel<<<blocksPerGrid, threadsPerBlock, sharedMemorySize>>>(d_graph, n, minWeight, maxWeight, time(NULL));

    if (cudaDeviceSynchronize() != cudaSuccess) {
        std::cerr << "Errore durante l'esecuzione del kernel." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Copia i dati dalla GPU alla CPU
    if (cudaMemcpy(graph, d_graph, size, cudaMemcpyDeviceToHost) != cudaSuccess) {
        std::cerr << "Errore nella copia dei dati dalla GPU alla CPU." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Libera memoria GPU
    cudaFree(d_graph);
}


int main() {
    int n = 1024; // Numero di nodi
    int start = 0; // Nodo iniziale
    int minWeight = 1;
    int maxWeight = 5000;

    // Genera il grafo casuale
    std::vector<int> graph(n * n);
    generateGraphCUDA(graph.data(), n, minWeight, maxWeight);

    // Timer per versione seriale
    std::vector<int> serialDistances(n);
    auto startTime = std::chrono::high_resolution_clock::now();
    dijkstraSerialCUDA(graph.data(), n, start, serialDistances.data()); // Esegui versione seriale
    auto endTime = std::chrono::high_resolution_clock::now();
    long serialTime = std::chrono::duration_cast<std::chrono::milliseconds>(endTime - startTime).count();

    // Timer per versione parallela (CUDA)
    std::vector<int> cudaDistances(n);
    startTime = std::chrono::high_resolution_clock::now();
    dijkstraCUDA(graph.data(), n, start); // Esegui versione parallela
    endTime = std::chrono::high_resolution_clock::now();
    long cudaTime = std::chrono::duration_cast<std::chrono::milliseconds>(endTime - startTime).count();

    // Stampa dei tempi di esecuzione
    std::cout << "\n=== Risultati ===\n";
    std::cout << "Tempo Seriale: " << serialTime << " ms\n";
    std::cout << "Tempo CUDA: " << cudaTime << " ms\n";
    std::cout << "Differenza: " << (serialTime - cudaTime) << " ms\n";

    return 0;
}









Overwriting dijkstra_serialMerge.cu


In [ ]:
!ncu --mode launch-and-attach -o profile --target-processes all --nvtx --call-stack --section ComputeWorkloadAnalysis --section MemoryWorkloadAnalysis -f ./dijkstra_serialMerge.o


==PROF== Connected to process 11281 (/content/dijkstra_serialMerge.o)
==PROF== Profiling "generateGraphKernel" - 0: 0%....50%....100% - 8 passes
==PROF== Profiling "dijkstraSerialKernel" - 1: 0%....50%....100% - 8 passes
==PROF== Profiling "findClosestNodeCUDA" - 2: 0%..
==WARNING== Launching the workload is taking more time than expected. If this continues to hang, terminate the profile and re-try by profiling the range of all related launches using '--replay-mode range'. See https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#replay for more details.


In [ ]:
!nvcc -rdc=true dijkstra_serialMerge.cu -lcudadevrt -o dijkstra_serialMerge.o

In [ ]:
!ncu --target-processes=all ./dijkstra_serialMerge.o